# 前言 : 

1.1.0 是对于QA_Market的一个小规模重构,区别以往的订单,成交模式, 故更新一个小版本

本次的更新主要有:


[QAData]

1. 修改了采样函数的写法
2. 修复了tick采样成60min的bug
3. 修复了因为multiindex导致的QA_DataStruct.to_json方法缺失 datetime/code 字段的问题

[QAMARKET]


1. 增加对于实盘易的支持
2. 将一些基础解析字段挪至基类QABroker中
3. 基于QAMarket创建的broker/order线程全部变成后台线程
4. QA_OrderHandler 增加持久化 订单/成交单部分
5. 修改QAOrder 变成有限状态机, 通过状态机的动作自动改变Order的状态
6. 修改QABacktest_Broker 适配新的order类
7. 优化QABroker的状态显示
8. 将撮合缓存字典重构进QADealer, dealer加入 deal_message(dict) 和deal_df(dataframe),并增加dealer的settle函数
9. 大量更新QABacktest_Broker方法,使之适配新版本回测
10. 适配实盘,增加字段解析 trade_towards_cn_en, order_status_cn_en
11. 修改order的撮合机制, 使用创建order时的callback回调来更新账户


[QAEngine]

1. QAThread 初始化增加 daemon选项, 用于创建守护线程

[QAARP]

1. QA_Account 增加 cancel_order方法 撤单操作
2. 重写QA_Account的receive_deal方法, 适配新版本更新账户
3. 优化和修改QA_Portfolio 增加对于history/history_table的支持


[QASU]

1. save_orderhandler.py 增加 QA_SU_save_order/ QA_SU_save_deal 方法
2. 增加对于运行时出错的容错处理

[QAUtil]

1. QADate_Trade 增加QA_util_get_order_datetime() 用于获取委托的真实日期 QA_util_get_trade_datetime() 用于获取成交的真实日期
2. QA_Parameter 修改订单状态

[QAWEB]

1. QAWEB 增加查询股票名称的接口 http://ip:port/marketdata/stock/code?code=xxxxx

In [1]:
import QUANTAXIS as QA
import pandas as pd
import threading
import time
import datetime
import numpy as np

QUANTAXIS>> start QUANTAXIS
QUANTAXIS>> Welcome to QUANTAXIS, the Version is 1.1.0
QUANTAXIS>>  
 ```````````````````````````````````````````````````````````````````````````````````````````````````````````````````````` 
  ``########`````##````````##``````````##`````````####````````##```##########````````#``````##``````###```##`````######`` 
  `##``````## ```##````````##`````````####````````##`##```````##```````##```````````###``````##````##`````##```##`````##` 
  ##````````##```##````````##````````##`##````````##``##``````##```````##``````````####```````#```##``````##```##``````## 
  ##````````##```##````````##```````##```##```````##```##`````##```````##`````````##`##```````##`##```````##````##``````` 
  ##````````##```##````````##``````##`````##``````##````##````##```````##````````##``###```````###````````##`````##`````` 
  ##````````##```##````````##``````##``````##`````##`````##```##```````##```````##````##```````###````````##``````###```` 
  ##````````##```##````````##`````##``````

In [2]:
if QA.__version__<'1.1.0':
    print('请升级quantaxis后再运行此教程')

# 首先我们介绍QA_MARKET:

1. 如何理解QA_MARKET:
    QA_MARKET 是一个交易前置, 从使用的角度看,你可以把他理解成一个交易终端(比如同花顺/通达信)
    
    在去年准备对于MARKET类重构的时候,QA_MARKET的作用图如下:
    
    ![pic](http://pic.yutiansut.com/%E9%87%8D%E6%9E%84%E6%96%87%E6%A1%A3-%E5%B8%82%E5%9C%BA.png)
    
    
    交易前置主要起的作用是:
    
    1. 向上接入不同的Broker
    2. 向下接入不同的账户
    
    broker可以利用不同的接口, 接入不同的市场(股票/期货/港股/美股/模拟盘/回测)

2. QA_MARKET的类构成:

    QA_Market继承于 QA_Trade, QA_Trade 是一个交易前置基类, 其中包含了 QA_Engine(用于开启多线程的管理引擎), 以及各种基类方法

3. QA_Market的主要方法:

    - start() 启动交易前置
    
    - connect(broker) 连接broker  (等同于在同花顺里面选择不同的券商)
    
    - register(broker_name,broker) 注册一个broker(如果不是官方支持的broker)
    
    - login(account_cookie,account) 登陆账户
    
    - login_out(account_cookie,account) 登出
    
    - get_account(account_cookie) 获得账户句柄
    
    - insert_order(account_cookie, amount, amount_model, time, code, price, order_model, towards, market_type, frequence, broker_name,     money=None)  插入订单
    
    - cancel_order(broker_name,account_cookie,order_id) 撤单
    
    - cancel_all(broker_name,account_cookie) 某一个账户订单全撤
    
    - query_assets(account_cookie) 查询某个账户的资产
    
    - query_position(account_cookie) 查询某个账户的持仓
    
    - query_cash(account_cookie) 查询某个账户的现金
    
    - query_data(broker_name, frequence, market_type, code, start, end=None) 查询历史行情数据
    
    - query_currentbar(broker_name, market_type, code) 查询当前行情
    
    - get_trading_day() 查询当前交易日期
    
    
4. QA_Market的非阻塞事件

    QA_Market的非阻塞事件由 QA_Engine, Engine 中管理着各个QA_Thread, 通过submit(QA_Task) 和submit_nowait(QA_Task)的方法将事件推送到处理队列中
    
    - upcoming_data(broker, data) 行情推送进交易前置, 分发至market下的所有账户中
    
    - settle(broker) 每日交易结算
    
    - insert_order 插入订单 可以在on_insert_order中写异步处理(需要继承)
    
    关于submit函数和QA_Task
    
    ![pic](http://pic.yutiansut.com/QUANTAXISEvent.png)
    
    你可以在QA_Task的QA_Event中自行写入回调来异步调用你的函数
    
    
    
    

    
    
    
    
    